In [1]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import os
from imutils import paths


In [2]:
# Load images
path_dir = r"C:\Hope_Learning\Deep_learning\Supervised_learning\Eye_Open_or_Closed_Detection"
print("Loading Images From File Directory...")
imgs_path = list(paths.list_images(path_dir))

Loading Images From File Directory...


In [3]:
data = []
labels = []

# Preprocess images and labels
for img_path in imgs_path:
    label = img_path.split(os.path.sep)[-2]  # Get label from the directory name
    labels.append(label)

    image = load_img(img_path, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)  # Preprocess for MobileNetV2
    data.append(image)


In [4]:
print(f"Data Count:{len(data)}")
print(f"labels Count:{len(labels)}")

Data Count:3574
labels Count:3574


In [5]:
print(f"Eye Open Count:{labels.count('Opened_Eye')}")
print(f"Eye Close Count:{labels.count('Closed_Eye')}")

Eye Open Count:1802
Eye Close Count:1772


In [6]:
print(f"Label :{labels[0]}")
print(f"Label :{labels[1772]}")


Label :Closed_Eye
Label :Opened_Eye


In [7]:
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [8]:
print(data[0])

[[[0.8980392  0.9764706  0.9137255 ]
  [0.8980392  0.9764706  0.9137255 ]
  [0.8980392  0.9764706  0.9137255 ]
  ...
  [0.9137255  0.96862745 0.90588236]
  [0.9137255  0.96862745 0.90588236]
  [0.9137255  0.96862745 0.90588236]]

 [[0.8980392  0.9764706  0.9137255 ]
  [0.8980392  0.9764706  0.9137255 ]
  [0.8980392  0.9764706  0.9137255 ]
  ...
  [0.9137255  0.96862745 0.90588236]
  [0.9137255  0.96862745 0.90588236]
  [0.9137255  0.96862745 0.90588236]]

 [[0.8980392  0.9764706  0.9137255 ]
  [0.8980392  0.9764706  0.9137255 ]
  [0.8980392  0.9764706  0.9137255 ]
  ...
  [0.9137255  0.96862745 0.90588236]
  [0.9137255  0.96862745 0.90588236]
  [0.9137255  0.96862745 0.90588236]]

 ...

 [[0.9137255  0.96862745 0.90588236]
  [0.9137255  0.96862745 0.90588236]
  [0.9137255  0.96862745 0.90588236]
  ...
  [0.9372549  0.9607843  0.8745098 ]
  [0.9372549  0.9607843  0.8745098 ]
  [0.9372549  0.9607843  0.8745098 ]]

 [[0.9137255  0.96862745 0.90588236]
  [0.9137255  0.96862745 0.90588236]


In [9]:
print(labels[0])
print(labels[1772])

Closed_Eye
Opened_Eye


In [10]:
# Label encoding
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [11]:
print(labels[0])
print(labels[1772])

[1. 0.]
[0. 1.]


In [12]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)

In [13]:
pre_trained_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Add new layers on top of the pre-trained model
join_our_model = pre_trained_model.output
join_our_model = AveragePooling2D(pool_size=(2, 2))(join_our_model)
join_our_model = Flatten(name="flatten")(join_our_model)
join_our_model = Dense(128, activation="relu")(join_our_model)
join_our_model = Dropout(0.5)(join_our_model)
join_our_model = Dense(2, activation="softmax")(join_our_model)

# Create the final model
model = Model(inputs=pre_trained_model.input, outputs=join_our_model)


In [14]:
for layer in pre_trained_model.layers:
    layer.trainable = False

# Compile model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(X_train, y_train, batch_size=100,validation_data=(X_test, y_test),epochs=5)



Epoch 1/5


29/29 [==============================] - 34s 1s/step - loss: 0.3898 - accuracy: 0.8937 - val_loss: 0.0558 - val_accuracy: 0.9874
Epoch 2/5
29/29 [==============================] - 30s 1s/step - loss: 0.0516 - accuracy: 0.9860 - val_loss: 0.0218 - val_accuracy: 0.9930
Epoch 3/5
29/29 [==============================] - 30s 1s/step - loss: 0.0260 - accuracy: 0.9923 - val_loss: 0.0149 - val_accuracy: 0.9958
Epoch 4/5
29/29 [==============================] - 30s 1s/step - loss: 0.0117 - accuracy: 0.9979 - val_loss: 0.0121 - val_accuracy: 0.9972
Epoch 5/5
29/29 [==============================] - 30s 1s/step - loss: 0.0098 - accuracy: 0.9979 - val_loss: 0.0101 - val_accuracy: 0.9972


In [15]:
y_pred=model.predict(X_test)

23/23 [==============================] - 8s 287ms/step


In [16]:
ypredarg=np.argmax(y_pred,axis=1)

In [19]:
ypredarg

array([1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,

In [20]:
y_test

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [21]:
ytestarg=np.argmax(y_test,axis=1)

In [22]:
ytestarg

array([1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,

In [23]:
CR=classification_report(ytestarg,ypredarg)

In [24]:
CR

'              precision    recall  f1-score   support\n\n           0       1.00      0.99      1.00       354\n           1       0.99      1.00      1.00       361\n\n    accuracy                           1.00       715\n   macro avg       1.00      1.00      1.00       715\nweighted avg       1.00      1.00      1.00       715\n'

In [25]:
model.save('Eye_Detection_Model.h5')

C:\Users\Nagendran\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
